In [ ]:
!pip install geemap

import ee
import geemap

In [ ]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

In [ ]:
# Takes an image collection and returns a collection with monthly means
def col2monthly(col):
    
    def monthly(m):
        return (col.filter(ee.Filter.calendarRange(y_start, y_end, 'year'))
                .filter(ee.Filter.calendarRange(m, m, 'month')).mean()
                .set('month', m)
                .set('ID', ee.Number(m).format('%02d'))
                )

    
    months = ee.List.sequence(1, 12)
    col_m = ee.ImageCollection.fromImages(months.map(monthly).flatten())
    
    return col_m

def grid2scale(img):
    return img.reproject(crs='EPSG:4326', scale=scale)

# Parameters for computation
y_start = 2001
y_end = 2020
scale = 11100


In [ ]:
# Define climate variables to use for clustering
# gives mm/h
p = col2monthly(ee.ImageCollection("NASA/GPM_L3/IMERG_MONTHLY_V06").select('precipitation')).map(grid2scale)

# gives K
t = col2monthly(ee.ImageCollection("ECMWF/ERA5_LAND/MONTHLY").select('temperature_2m')).map(grid2scale)

# Define region of interest (roi)
roi = ee.FeatureCollection("USDOS/LSIB_SIMPLE/2017").filter(ee.Filter.eq('country_co', 'BR'))

In [ ]:
p.getInfo()

In [ ]:
# Let's get the monthly data and calculate the following inputs:
# annual mean monthly precipitation (P_ann),
# annual mean LST (T_ann),
# highest and lowest monthly precipitation (P_max, P_min)
# highest and lowest monthly temperature (T_max, T_min)

p_mean = p.mean().multiply(24*30).rename('p_avg')
p_max = p.max().multiply(24*30).rename('p_max')
p_min = p.min().multiply(24*30).rename('p_min')

t_mean = t.mean().subtract(273).rename('t_avg')
t_max = t.max().subtract(273).rename('t_max')
t_min = t.min().subtract(273).rename('t_min')

img_input = p_mean.addBands([p_max, p_min, t_mean, t_max, t_min]) # Lista com tudo

In [ ]:
Map = geemap.Map()

# Display the mean P and T
Map.addLayer(img_input, {}, 'climate_vars')
Map

In [ ]:
import geemap.colormaps as gcm

cmp = gcm.palettes
parvis = {'min': 15, 'max': 30, 'palette': cmp.viridis_r}

lyrname = 'Average monthly temperature'
Map.addLayer(t_mean, parvis, lyrname)
Map.add_colorbar(parvis, label = None, discrete=False, orientation='vertical', layer_name = lyrname)

In [ ]:
# Make the training dataset.
training = img_input.sample(
    region = roi,
    scale = scale,
    numPixels = 10000
)

# Instantiate the clusterer and train it.
clusterer = ee.Clusterer.wekaKMeans(9).train(training)

# Cluster the input using the trained clusterer.
result = img_input.cluster(clusterer).clip(roi)

In [ ]:
Map = geemap.Map(center=[-14, -55], zoom=4)

# Koppen climate zones
czns = (ee.FeatureCollection('projects/et-brasil/assets/Climate_Koppen_Brasil')
    .reduceToImage(properties=['GRIDCODE'], reducer=ee.Reducer.first())
    )

# Display the clusters with random colors.
Map.addLayer(result.randomVisualizer(), {}, 'Climate clusters K-means')
Map.addLayer(czns.randomVisualizer(), {}, 'Koppen Climate Zones')
Map

In [ ]:
Map = geemap.Map(center=[-14, -55], zoom=4)

l_lyr = geemap.ee_tile_layer(result.randomVisualizer(), {}, 'Climate clusters K-means')
r_lyr = geemap.ee_tile_layer(czns.randomVisualizer(), {}, 'Koppen Climate Zones')

Map.split_map(left_layer=l_lyr, right_layer=r_lyr)
Map

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

geemap.linked_maps(rows=1, cols=2, ee_objects=[result.randomVisualizer(), czns.randomVisualizer()], 
                   center=[-14, -55], zoom=4,
                   vis_params=[{}, {}], labels=['1', '2'])